In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
import os
import sys
sys.path.append(os.path.abspath("/Users/mingl/Desktop/info6120/group_project/TouchInput_Exploration-main/data_preparation"))
from plot_profiles import plot_profiles_split_channels, plot_profiles
from load_audio import load_audio


In [13]:
path = "/Users/mingl/Desktop/info6120/group_project/audios/session_01"

os.chdir(path)

config_path = path + "/config.json"

In [14]:
# Get config file
audio_config = json.load(open(config_path, 'rt'))

In [16]:
echo_profile_paths = []
diff_profile_paths = []
frame_time_paths = []
record_paths = []
video_paths = []

for audio_file, gt_file, video_file in zip(audio_config["audio"]["files"], audio_config["ground_truth"]["files"], audio_config["ground_truth"]["videos"]):
    echo_profile_paths.append(path + "/" + audio_file.split(".")[0] + "_fmcw_16bit_profiles.npy")
    diff_profile_paths.append(path + "/" + audio_file.split(".")[0] + "_fmcw_16bit_diff_profiles.npy")
    record_paths.append(path + "/" + gt_file)
    frame_time_paths.append(path + "/" + gt_file.split("records")[0] + "frame_time.txt")
    video_paths.append(path + "/" + video_file)

In [17]:
session_num = 0

echo_profile_path = echo_profile_paths[session_num]
diff_profile_path = diff_profile_paths[session_num]
frame_time_path = frame_time_paths[session_num]
record_path = record_paths[session_num]
video_path = video_paths[session_num]

print(echo_profile_path)
print(diff_profile_path)
print(frame_time_path)
print(record_path)
print(video_path)

/Users/mingl/Desktop/info6120/group_project/audios/session_01/audio_001_fmcw_16bit_profiles.npy
/Users/mingl/Desktop/info6120/group_project/audios/session_01/audio_001_fmcw_16bit_diff_profiles.npy
/Users/mingl/Desktop/info6120/group_project/audios/session_01/record_20250429_044416_327788_frame_time.txt
/Users/mingl/Desktop/info6120/group_project/audios/session_01/record_20250429_044416_327788_records.txt
/Users/mingl/Desktop/info6120/group_project/audios/session_01/record_20250429_044416_327788.mp4


In [ ]:
# Read record file
record_start_frames = []
record_start_poses = []
record_end_poses = []
record_classes = []

with open(record_path, 'r') as record_file: 
    with open(frame_time_path, 'r') as frame_time_file: 
        records = record_file.readlines()
        frame_times = frame_time_file.readlines()
        frame_time_line_count = 0
        for i in range(len(records)):
            curr_record_line = records[i]
            record_start_time = float(curr_record_line.split(",")[1])
            record_end_time = float(curr_record_line.split(",")[2])
            while record_start_time > float(frame_times[frame_time_line_count]):
                frame_time_line_count += 1
            record_start_frame = frame_time_line_count + 1
            record_start_frames.append(record_start_frame)
            record_start_pos = round((record_start_time - float(frame_times[audio_config['ground_truth']['syncing_poses'][session_num]])) * audio_config['audio']['config']['sampling_rate'] / audio_config['audio']['config']['frame_length'] + audio_config['audio']['syncing_poses'][session_num])
            record_end_pos = round((record_end_time - float(frame_times[audio_config['ground_truth']['syncing_poses'][session_num]])) * audio_config['audio']['config']['sampling_rate'] / audio_config['audio']['config']['frame_length'] + audio_config['audio']['syncing_poses'][session_num])
            record_start_poses.append(record_start_pos)
            record_end_poses.append(record_end_pos)
            #modified here 
            record_classes.append(curr_record_line.split(",")[3].split("-")[-1].rstrip())
        frame_time_file.close()
    record_file.close()

In [19]:
# Get session start time
session_start_time = audio_config["sessions"][session_num][0]["start"]
session_start_pos = round((session_start_time - float(frame_times[audio_config['ground_truth']['syncing_poses'][session_num]])) * audio_config['audio']['config']['sampling_rate'] / audio_config['audio']['config']['frame_length'] + audio_config['audio']['syncing_poses'][session_num])

In [20]:
# Get echo profiles
echo_profiles = np.load(echo_profile_path)
diff_profiles = np.load(diff_profile_path)

echo_profile_img_bgr = plot_profiles_split_channels(echo_profiles, 4)

# bgr to rgb
echo_profile_img = echo_profile_img_bgr.copy()
echo_profile_img[:,:,0] = echo_profile_img_bgr[:,:,2]
echo_profile_img[:,:,2] = echo_profile_img_bgr[:,:,0]

print(echo_profiles.shape)
print(echo_profile_img.shape)
# plt.imshow(echo_profile_img[:,:])

(2400, 49275)
(2420, 49275, 3)


In [ ]:
s1 = echo_profiles[:, record_start_pos[0]:record_end_poses[0]]
s2 = echo_profiles[:, record_start_pos[1]:record_end_poses[1]]
s3 = echo_profiles[:, record_start_pos[2]:record_end_poses[2]]
s4 = echo_profiles[:, record_start_pos[3]:record_end_poses[3]]
s5 = echo_profiles[:, record_start_pos[4]:record_end_poses[4]]
s6 = echo_profiles[:, record_start_pos[5]:record_end_poses[5]]
s7 = echo_profiles[:, record_start_pos[6]:record_end_poses[6]]
s8 = echo_profiles[:, record_start_pos[7]:record_end_poses[7]]
s9 = echo_profiles[:, record_start_pos[8]:record_end_poses[8]]


In [ ]:
# Crop echo profiles and put text based on labels

diff_profile_img_bgr = plot_profiles_split_channels(diff_profiles, 4, 30000000, -30000000)

# duration_frame = int((record_start_frames[max_idx + 1] - record_start_frames[min_idx]) / (max_idx + 1 - min_idx))
for i in range(len(record_start_frames)):   # range(min_idx, max_idx + 1):
    # plt.text(i * duration_frame, 100, str(record_classes[i].split("_")[1].rstrip()), fontsize=5, bbox=dict(facecolor='cyan', edgecolor='cyan', linewidth=1))
    cv2.putText(diff_profile_img_bgr, str(record_classes[i]), (record_start_poses[i], 165), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 2)

# plt.imshow(diff_profile_img[0:600, record_start_frames[min_idx] : record_start_frames[max_idx + 1]])
cv2.imwrite("test.png", diff_profile_img_bgr)


True

In [1]:
diff_profiles = np.load("/Users/mingl/Desktop/info6120/group_project/audios/session_01/audio_001_fmcw_16bit_diff_profiles.npy")


assert len(record_start_poses) == len(record_end_poses) == len(record_classes)

# ── 3) Slice & label, dropping any empty/too-short segments ─────────────────
segments = []
labels   = []

lengths = np.array([seg.shape[0] for seg in segments])
target_len = int(np.percentile(lengths, 90))

# 2. (Optional) down-sample
downsample = 5
segments = [seg[::downsample] for seg in segments]

# 3. Pad or truncate
padded = []
for seg in segments:
    T, C = seg.shape
    if T < target_len:
        pad_w = ((0, target_len - T), (0, 0))
        seg2 = np.pad(seg, pad_w, "constant", constant_values=0)
    else:
        seg2 = seg[:target_len]
    padded.append(seg2)

X = np.stack(padded)           # → (N, target_len, C)
y = np.array(labels)

np.save("X2.npy", X)   # X has shape (N, T, C, 1)
np.save("y2.npy", y)   # y has shape (N,)

NameError: name 'np' is not defined

In [33]:
# 1) Load your long differential echo‐profile array
#    (shape = [T, C] where T = total time‐samples, C = channels)
diff_profiles = np.load("/Users/mingl/Desktop/info6120/group_project/audios/session_01/audio_001_fmcw_16bit_diff_profiles.npy")

# 2) Re-create the lists your snippet already built:
#    – record_start_poses: the sample index where each clap (start of a gesture) occurs
#    – record_classes:      the string labels, e.g. ["pos_0","pos_0","pos_1",…]
#    (If you populated these in your notebook, you can just import them here.)
#
# For demonstration let’s assume you already have:
#    record_start_poses = [...]
#    record_classes     = [...]
#
# Convert your class strings to integer IDs:
assert len(record_start_poses) == len(record_end_poses) == len(record_classes)

# ── 3) Slice & label, dropping any empty/too-short segments ─────────────────
segments = []
labels   = []

for i, (start, end, cls_str) in enumerate(zip(record_start_poses,
                                               record_end_poses,
                                               record_classes)):
    length = end - start
    if length <= 0:
        print(f"Skipping segment {i}: start={start}, end={end}, length={length}, class={cls_str}")
        continue

    seg = diff_profiles[start:end, :]   # → (length, channels)
    segments.append(seg)
    labels.append(int(cls_str))   # "pos_3" → 3

# Quick sanity check
print(f"\nKept {len(segments)} / {len(record_classes)} segments")

# ── 4) Pad or truncate each segment to fixed time-length ───────────────────────
#    You can choose a target_len (e.g. max or some window)
lengths   = [s.shape[0] for s in segments]
target_len = max(lengths)   # or set target_len = 200, etc.

padded = []
for seg in segments:
    T, C = seg.shape
    if T < target_len:
        pad_w = ((0, target_len - T), (0, 0))
        seg2 = np.pad(seg, pad_w, mode="constant", constant_values=0)
    else:
        seg2 = seg[:target_len, :]
    padded.append(seg2)

X = np.stack(padded, axis=0)            # → (N, target_len, channels)
y = np.array(labels, dtype=np.int64)    # → (N,)

# ── 5) (Optional) add channel-axis for a 2D-CNN ──────────────────────────────
X = X[..., np.newaxis]                  # → (N, target_len, channels, 1)

print("Final shapes:", X.shape, y.shape)



Kept 72 / 72 segments
Final shapes: (72, 640, 49274, 1) (72,)


In [ ]:
np.save("X.npy", X)   # X has shape (N, T, C, 1)
np.save("y.npy", y)   # y has shape (N,)

: 

In [ ]:
data_folder = "/Users/mingl/Desktop/info6120/group_project/model"
profiles = np.load(os.path.join(data_folder,
                                "audio_001_fmcw_16bit_diff_profiles.npy"))
segments = []
labels   = []

ends = record_start_poses[1:] + [profiles.shape[0]]
for cls_str, start, end in zip(record_classes, record_start_poses, ends):
    seg = profiles[start:end, :]               # now end>start
    segments.append(seg)
    labels.append(int(cls_str))

# 3c. stack into arrays
#     Depending on your model you may need to add a “channel” axis:
for i, seg in enumerate(segments):
    print(f"Segment {i:2d}: shape={seg.shape}")
X = np.stack(segments, axis=0)           # → (N, duration, channels)
y = np.array(labels, dtype=np.int64)     # → (N,)

print(f"Built X.shape={X.shape}, y.shape={y.shape}, labels range {y.min()}–{y.max()}")

# ── 4. Save for downstream training ───────────────────────────────────────────
np.save(os.path.join(data_folder, "X.npy"), X)
np.save(os.path.join(data_folder, "y.npy"), y)

Final X.shape: (2, 712, 49274)
Built X.shape=(2, 712, 49274), y.shape=(2,), labels range 1–1


In [14]:
# Reorder based on classes and save
def save_reorder_profiles(original_profiles, reordered_img_name, minval, maxval, record_classes, record_start_poses, record_end_poses):
    reordered_profiles = np.zeros(original_profiles.shape)
    done_classes = []
    curr_profile_pos = 0
    for curr_class in record_classes:
        if curr_class not in done_classes:
            for i in range(len(record_classes)):
                if record_classes[i] == curr_class:
                    buffer = (record_end_poses[curr_profile_pos] - record_start_poses[curr_profile_pos]) - (record_end_poses[i] - record_start_poses[i])
                    reordered_profiles[:, record_start_poses[curr_profile_pos] + buffer : record_end_poses[curr_profile_pos]] = original_profiles[:, record_start_poses[i] : record_end_poses[i]]
                    done_classes.append(curr_class)
                    curr_profile_pos += 1

    reordered_profile_img_bgr = plot_profiles_split_channels(reordered_profiles, 2, maxval, minval)
    for i in range(len(done_classes)):   
        cv2.putText(reordered_profile_img_bgr, str(done_classes[i]), (record_start_poses[i], 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 2)
    cv2.imwrite(reordered_img_name, reordered_profile_img_bgr)

    return reordered_profiles
    


In [28]:
# Reorder based on classes
def reorder_profiles(original_profiles, record_classes, record_start_poses, record_end_poses):
    reordered_profiles = np.zeros(original_profiles.shape)
    reordered_classes = []
    reordered_start_poses = []
    reordered_end_poses = []
    curr_profile_pos = 0
    for curr_class in record_classes:
        if curr_class not in reordered_classes:
            for i in range(len(record_classes)):
                if record_classes[i] == curr_class:
                    buffer = (record_end_poses[curr_profile_pos] - record_start_poses[curr_profile_pos]) - (record_end_poses[i] - record_start_poses[i])
                    reordered_profiles[:, record_start_poses[curr_profile_pos] + buffer : record_end_poses[curr_profile_pos]] = original_profiles[:, record_start_poses[i] : record_end_poses[i]]
                    reordered_classes.append(curr_class)
                    reordered_start_poses.append(record_start_poses[curr_profile_pos])
                    reordered_end_poses.append(record_end_poses[curr_profile_pos])
                    curr_profile_pos += 1

    return reordered_profiles, reordered_classes, reordered_start_poses, reordered_end_poses